In [1]:
from glob import glob
from pathlib import Path
import shutil
from pdb import set_trace
import pandas as pd

In [2]:
files = glob("data/*")

In [3]:
group = {}
for i in files:
    types = i.split("/")[-1].split("_x_")[0]
    if types not in group:
        group[types] = [i]
    else:
        group[types].append(i)
len(group)

4

In [13]:
Path("processed").mkdir(exist_ok=True)

In [14]:
for k in group:
    if k == 'dosm-public-mets':
        s = group[k]
        lookups = [pd.read_csv(i,index_col=0) for i in s if "lookup_hs" in i][0]
        sources = pd.concat([pd.read_csv(i,index_col=0) for i in s if "lookup" not in i])
        mets_df = pd.merge(sources, lookups, left_on = 'hs_6d', right_on = 'hs_6d', how = 'left').drop('hs_6d', axis = 1)
        mets_df.to_csv("processed/mets.csv",index=False)
    elif k == 'dosm-public-pricecatcher':
        s = group[k]
        item = pd.read_csv("data/dosm-public-pricecatcher_x_lookup_item_preview.parquet",index_col=0)
        premise = pd.read_csv("data/dosm-public-pricecatcher_x_lookup_premise_preview.parquet",index_col=0)
        sources = pd.concat([pd.read_csv(i,index_col=0) for i in s if "lookup" not in i])
        left = pd.merge(sources, item, left_on = 'item_code', right_on = 'item_code', how = 'left').drop('item_code', axis = 1)
        price = pd.merge(left, premise, left_on = 'premise_code', right_on = 'premise_code', how = 'left').drop('premise_code', axis = 1)
        mets_df.to_csv("processed/price.csv",index=False)
    else:
        source = group[k]
        for i in source:
            targ = i.replace("data","processed").replace(".parquet",".csv")
            if Path(targ).is_file():
                continue
            df = pd.read_csv(i,index_col=0)
            df.to_csv(targ,index=False)